# A LangChain Bot

In [1]:
import sys
sys.path.insert(1, '../../')
import init_creds as creds
 
AZURE_OPENAI_KEY = creds.get_api_key()
AZURE_OPENAI_ENDPOINT = creds.get_endpoint()
 
if not AZURE_OPENAI_KEY:
    raise ValueError("No AZURE_OPENAI_KEY set for Azure OpenAI API")
if not AZURE_OPENAI_ENDPOINT:
    raise ValueError("No AZURE_OPENAI_ENDPOINT set for Azure OpenAI API")

In [2]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    model="gpt-4o-mini",
    api_version="2024-07-01-preview"    
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [4]:
from typing import Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [6]:
config = {"configurable": {"thread_id": "Conversation00001"}}

In [7]:
from langchain_core.messages import trim_messages

trimmer = trim_messages(
    max_tokens=100000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [8]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph

# Async function for node:
async def call_model(state: State, config: dict):
    chain = prompt | trimmer | model | parser
    response = await chain.ainvoke(state, config)
    return {"messages": [response]}

# Define graph as before:
workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

In [11]:
from langchain_core.messages import HumanMessage, AIMessage

async def test_chat(query):
    message = HumanMessage(query)
    input_messages = [message]
    async for chunk, metadata in app.astream(input={"messages": input_messages}, config=config, stream_mode="messages"):
        if isinstance(chunk, AIMessage):
            yield chunk.content

async for content in test_chat("Hi there, my name is Bill, and I work in a button factory."):
    print(content)

# await test_chat("One Day")
# await test_chat("My boss comes up to me.")
# await test_chat("He says, Hey Bill, whatcha doin'?")    
# await test_chat("I said, nuthin'")  
# await test_chat("He said, Well then, do this! And he started moving his hands around in big circles.")   
# await test_chat("So I did.")


Ah
oy
 again
,
 Bill
!
 Ye
 be
 a
 persistent
 lad
,
 I
 see
!
 A
 button
 factory
,
 ye
 say
?
 What
 kinds
 o
’
 buttons
 do
 ye
 make
?
 Be
 they
 for
 shirts
,
 trousers
,
 or
 perhaps
 som
eth
in
’
 more
 fancy
 like
 pirate
 coats
?
 Speak
 up
,
 mate
y
!

